## Part 0: Data Collection from Google Big Query

### Add authentication credentials to access Big Query API

In [1]:
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/psparks/Documents/Springboard/GDELT Testing-31a7081b0cbc.json"

### The BigQuery Python client library provides a magic command that allows you to run queries with minimal code. To load the magic commands from the client library, paste the following code into the first cell of the notebook.

In [2]:
# %load_ext google.cloud.bigquery

### The BigQuery client library provides a cell magic,  _%%bigquery_, which runs a SQL query of all French BigQuery events since Feb. 1 2016 and returns the results as a Pandas DataFrame.

In [3]:
# %%bigquery france_subset
# SELECT *
# FROM `gdelt-bq.full.events`
# WHERE ActionGeo_CountryCode = 'FR' AND MonthYear >= 201601

In [4]:
# france_subset.shape

### After downloading data, save it for future use

In [5]:
# france_subset.to_csv('france_subset_1619_2.csv.bz2', header = True, compression='bz2')

### Now read it back for future data wrangling

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import warnings; warnings.simplefilter('ignore')

In [4]:
%time # ~ 6mins
france_subset = pd.read_csv('france_subset_1619_2.csv.bz2', compression='bz2')

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 14.1 µs


In [5]:
france_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103885 entries, 0 to 3103884
Data columns (total 59 columns):
Unnamed: 0               int64
GLOBALEVENTID            int64
SQLDATE                  int64
MonthYear                int64
Year                     int64
FractionDate             float64
Actor1Code               object
Actor1Name               object
Actor1CountryCode        object
Actor1KnownGroupCode     object
Actor1EthnicCode         object
Actor1Religion1Code      object
Actor1Religion2Code      object
Actor1Type1Code          object
Actor1Type2Code          object
Actor1Type3Code          object
Actor2Code               object
Actor2Name               object
Actor2CountryCode        object
Actor2KnownGroupCode     object
Actor2EthnicCode         object
Actor2Religion1Code      object
Actor2Religion2Code      object
Actor2Type1Code          object
Actor2Type2Code          object
Actor2Type3Code          object
IsRootEvent              int64
EventCode                int

## Part 1: Data Cleaning

### Let review what data we have.

In [6]:
france_subset.head()

,Unnamed: 0,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,0,523961397,20160324,201603,2016,2016.230100,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160324,http://www.goldcoastbulletin.com.au/news/natio...
1,1,521622559,20160316,201603,2016,2016.208200,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160316,http://www.newsmax.com/TheWire/italy-unsold-fo...
2,2,516418595,20160229,201602,2016,2016.161621,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160229,http://www.irishtimes.com/life-and-style/peopl...
3,3,522928257,20160321,201603,2016,2016.221900,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160321,http://www.ibtimes.co.uk/malaysia-palm-oil-cou...
4,4,538837301,20160510,201605,2016,2016.356200,NaN,NaN,NaN,NaN,...,-1409795,4,"French Riviera, France (general), France",FR,FR00,43.5599,7.13672,-1409795,20160510,http://movies.ndtv.com/bollywood/en-route-cann...


### We are going to subset the data based on the most relevant columns, according to source: https://jameshoward.us/wp-content/uploads/2018/03/Parrish-Crystal-Cube.pdf

In [7]:
france_events = france_subset[['SQLDATE', 'Actor1Name', 'Actor2Name', 'SOURCEURL', 'MonthYear', 'EventRootCode', 'GoldsteinScale', 'NumMentions', 'AvgTone', 'ActionGeo_CountryCode', 'ActionGeo_Lat', 'ActionGeo_Long']]

In [8]:
france_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103885 entries, 0 to 3103884
Data columns (total 12 columns):
SQLDATE                  int64
Actor1Name               object
Actor2Name               object
SOURCEURL                object
MonthYear                int64
EventRootCode            int64
GoldsteinScale           float64
NumMentions              int64
AvgTone                  float64
ActionGeo_CountryCode    object
ActionGeo_Lat            float64
ActionGeo_Long           float64
dtypes: float64(4), int64(4), object(4)
memory usage: 284.2+ MB


### Convert SQLDATE to datetime format

In [9]:
france_events.SQLDATE = pd.to_datetime(france_subset.SQLDATE, format = '%Y%m%d')

In [10]:
france_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103885 entries, 0 to 3103884
Data columns (total 12 columns):
SQLDATE                  datetime64[ns]
Actor1Name               object
Actor2Name               object
SOURCEURL                object
MonthYear                int64
EventRootCode            int64
GoldsteinScale           float64
NumMentions              int64
AvgTone                  float64
ActionGeo_CountryCode    object
ActionGeo_Lat            float64
ActionGeo_Long           float64
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 284.2+ MB


In [11]:
#france_events.sort_values(by=['SOURCEURL'], ascending=True).iloc[:5]

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
334888,2016-02-23,MINIST,PARIS,"5 Kanal TV, Kiev/BBC Monitoring/(c) BBC",201602,4,1.0,1,-0.302115,FR,49.0000,0.00000
1571981,2016-02-23,UKRAINE,GOVERNMENT OFFICIAL,"5 Kanal TV, Kiev/BBC Monitoring/(c) BBC",201602,4,7.0,1,3.021148,FR,49.0000,0.00000
1573287,2016-02-23,NaN,PARIS,"5 Kanal TV, Kiev/BBC Monitoring/(c) BBC",201602,4,1.0,8,3.021148,FR,48.8667,2.33333
491420,2017-03-09,DIPLOMAT,FRANCE,"5 Kanal TV, Kiev/BBC Monitoring/(c) BBC",201703,4,1.0,6,2.272727,FR,49.0000,0.00000
2045042,2017-03-09,UKRAINIAN,FRANCE,"5 Kanal TV, Kiev/BBC Monitoring/(c) BBC",201703,4,1.0,1,2.272727,FR,48.8667,2.33333


In [12]:
france_events.sample(10)

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
2797311,2016-05-21,STUDENT,FRANCE,http://www.peacearchnews.com/national/38030041...,201605,1,0.0,96,-2.143074,FR,48.8667,2.33333
623583,2016-05-21,SECURITY FORCE,PRESIDENT,http://www.jewishaz.com/us_worldnews/world/fre...,201605,14,-6.5,3,-7.633588,FR,48.8667,2.33333
2747484,2016-05-03,EUROPEAN,NaN,http://consequenceofsound.net/2016/05/win-vip-...,201605,2,3.4,10,2.602230,FR,46.0000,2.00000
1371178,2017-03-29,EUROPE,FRANCE,http://www.ctvnews.ca/canada/vimy-ridge-canadi...,201703,17,-5.0,4,2.247191,FR,50.3724,2.81034
377671,2018-05-28,PROSECUTOR,FRANCE,https://www.thestar.com/news/world/2018/05/28/...,201805,1,0.0,11,0.004519,FR,48.4353,4.38201
1724893,2017-02-16,ROME,NaN,http://blogs.bl.uk/digitisedmanuscripts/2017/0...,201702,1,0.0,3,-0.496524,FR,48.8667,2.33333
236550,2018-11-08,RUSSIAN,SYRIA,https://www.al-monitor.com/pulse/originals/201...,201811,4,1.9,1,-0.931414,FR,48.8667,2.33333
1628926,2019-04-11,AFRICA,TUNIS,https://www.prnewswire.com/news-releases/afric...,201904,4,1.0,10,4.695652,FR,48.8667,2.33333
194538,2018-03-05,GOOGLE,FRANCE,https://www.tax-news.com/news/Frances_Le_Maire...,201803,19,-10.0,10,-3.311258,FR,46.0000,2.00000
548440,2019-05-22,FRENCH,CIVILIAN,https://www.theguardian.com/world/2019/may/22/...,201905,5,3.8,4,-5.605137,FR,48.8667,2.33333


### Notice how we can't see the full URLs yet? Let's try to change that.

In [12]:
pd.set_option('display.max_colwidth', -1)

In [13]:
france_events.sample(5, random_state=1)

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
2130769,2018-09-01,PRESIDENT,PARIS,https://www.goskagit.com/news/world/how-macron-won-trump-s-friendship-failed-to-influence-him/article_40161889-95f8-58f9-996e-f156d774a9d8.html,201809,4,1.9,16,2.119462,FR,48.8667,2.33333
1242414,2016-02-05,PARIS,TERRORIST,http://www.arabnews.com/news/875471,201602,20,-10.0,20,-3.047394,FR,48.8667,2.33333
2550644,2018-05-29,CANADA,COMMUNITY,http://www.yorktonnews.com/france-s-macron-to-meet-trudeau-visit-montreal-ahead-of-g7-summit-in-quebec-1.23317715,201805,1,0.0,2,0.847458,FR,48.8667,2.33333
2984274,2019-03-05,FRENCH,NaN,https://www.msn.com/en-sg/lifestyle/style/teeny-tiny-handbag-becomes-a-fashion-sensation/ar-BBUnjMk,201903,11,-2.0,318,1.216732,FR,48.8667,2.33333
1304530,2018-05-06,BRITAIN,UNITED KINGDOM,https://www.bignewsnetwork.com/news/257198553/trump-stirs-another-controversy-over-gun-laws-uk-fumes,201805,3,4.0,1,-5.350118,FR,48.8667,2.33333


In [15]:
france_events.sample(5, random_state=2)

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
1994159,2017-03-29,PARIS,PREMIER,https://www.360nobs.com/2017/03/zlatan-ibrahimovic-hints/,201703,4,2.8,6,1.694915,FR,48.8667,2.33333
2838338,2018-02-16,BUSINESS,NaN,http://www.prweb.com/releases/2018/02/prweb15214603.htm,201802,1,0.0,6,5.280528,FR,47.1672,0.24564
2577313,2017-06-29,NAVAL,NaN,http://carnegieindia.org/publications/?fa=71395,201706,2,3.0,5,3.324808,FR,48.8667,2.33333
272698,2017-11-29,PARIS,NEW YORK,http://www.aljazeera.com/indepth/opinion/slavery-walls-fortress-europe-171128094218944.html,201711,14,-6.5,6,-6.577480,FR,48.8667,2.33333
1743688,2019-06-10,JEWISH,FRENCH,https://harpers.org/blog/2019/06/my-french-side/,201906,5,3.5,2,-0.941423,FR,46.7333,2.70000


#### After grabbing the full URLs, we're able to see that the subjects in the stories match the actors listed in Actor1Name and Actor2Name.

### Those URLs could be cleaned up a bit more in order to group the news by domain name later.

In [14]:
domains = france_events['SOURCEURL'].str.extract('(?<=http://)(.*?)(?=/)|(?<=https://)(.*?)(?=/)')
domains = domains[0].fillna(domains[1]).fillna(france_events['SOURCEURL'])

In [15]:
domains[:20]

0     www.goldcoastbulletin.com.au
1     www.newsmax.com             
2     www.irishtimes.com          
3     www.ibtimes.co.uk           
4     movies.ndtv.com             
5     lompocrecord.com            
6     www.bbc.co.uk               
7     www.brisbanetimes.com.au    
8     www.bloomberg.com           
9     www.paxtonrecord.net        
10    www.bloomberg.com           
11    whoswholegal.com            
12    www.monitor.co.ug           
13    onmilwaukee.com             
14    www.sharenet.co.za          
15    www.thesudburystar.com      
16    listverse.com               
17    mobile.wnd.com              
18    ecommerce-journal.com       
19    www.enca.com                
Name: 0, dtype: object

In [16]:
france_events['DOMAINNAME'] = domains

In [17]:
france_events.head()

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,DOMAINNAME
0,2016-03-24,NaN,POLICE,http://www.goldcoastbulletin.com.au/news/national/qantas-pilot-believed-to-have-deliberately-crashed-plane-into-sea-off-byron-bay/news-story/2f1b1154e10c139a36312c088424f381?nk=ad15cd490373050ab7a2275d46ead101-1458786743,201603,1,0.0,84,-2.747776,FR,46.0000,2.00000,www.goldcoastbulletin.com.au
1,2016-03-16,NaN,LAWMAKER,http://www.newsmax.com/TheWire/italy-unsold-food-law/2016/03/16/id/719333/,201603,1,0.0,2,-0.643087,FR,46.0000,2.00000,www.newsmax.com
2,2016-02-29,NaN,IRELAND,http://www.irishtimes.com/life-and-style/people/life-and-death-the-rotunda-and-the-rising-1.2549489,201602,4,1.9,8,0.675454,FR,46.0000,2.00000,www.irishtimes.com
3,2016-03-21,NaN,FARMER,http://www.ibtimes.co.uk/malaysia-palm-oil-council-slams-frances-tax-palm-oil-vows-retaliation-1550662,201603,12,-4.0,10,-5.533493,FR,46.0000,2.00000,www.ibtimes.co.uk
4,2016-05-10,NaN,FRANCE,http://movies.ndtv.com/bollywood/en-route-cannes-mallika-sherawat-tweets-pic-of-the-eiffel-tower-1404752,201605,4,1.9,2,0.000000,FR,43.5599,7.13672,movies.ndtv.com


In [20]:
france_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103885 entries, 0 to 3103884
Data columns (total 13 columns):
SQLDATE                  datetime64[ns]
Actor1Name               object
Actor2Name               object
SOURCEURL                object
MonthYear                int64
EventRootCode            int64
GoldsteinScale           float64
NumMentions              int64
AvgTone                  float64
ActionGeo_CountryCode    object
ActionGeo_Lat            float64
ActionGeo_Long           float64
DOMAINNAME               object
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 307.8+ MB


### Of our remaining columns, which ones have missing values?

In [18]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

In [19]:
missing_zero_values_table(france_events)

Your selected dataframe has 13 columns and 3103885 Rows.
There are 3 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
Actor2Name,0,645666,20.8,645666,20.8,object
Actor1Name,0,212579,6.8,212579,6.8,object
GoldsteinScale,293127,52,0.0,293179,9.4,float64


 ### Adding the actor names may take a lot of web scraping and NLP processing that may be of dubious value at the moment, but having missing `GoldsteinScale` values would definitely skew our predictions later.

In [23]:
france_events[france_events['GoldsteinScale'].isnull()]

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,DOMAINNAME
10853,2016-03-17,DISSIDENT,RUSSIA,http://www.newstimes.com/news/world/article/Kazakh-banker-s-wife-urges-France-not-to-send-him-6893210.php,201603,12,NaN,1,-8.888889,FR,48.8667,2.33333,www.newstimes.com
61541,2019-03-19,FRANCE,RUSSIA,https://www.bbc.co.uk/news/world-asia-pacific-15483497,201903,12,NaN,8,-5.444646,FR,46.0000,2.00000,www.bbc.co.uk
69189,2016-03-22,NaN,FRANCE,http://www.ndtv.com/world-news/francois-hollande-meets-paris-attack-victims-after-suspect-captured-1289524,201603,12,NaN,30,-4.288813,FR,48.8667,2.33333,www.ndtv.com
76295,2016-03-16,FRANCE,RUSSIA,https://www.equities.com/news/kazakh-banker-s-wife-urges-france-not-to-send-him-to-russia,201603,12,NaN,42,-10.456544,FR,48.8667,2.33333,www.equities.com
192692,2016-03-21,LAWYER,FRANCE,http://www.ndtv.com/world-news/francois-hollande-to-meet-paris-attack-victims-after-suspect-captured-1289335,201603,12,NaN,16,-4.973868,FR,48.8667,2.33333,www.ndtv.com
264112,2016-03-09,FRANCE,THE NETHERLAND,http://www.msn.com/en-gb/news/uknews/dentist-of-horror-on-trial-for-mutilating-french-patients/ar-AAgxAJj,201603,12,NaN,2,-7.117438,FR,47.8632,1.80278,www.msn.com
266258,2016-03-21,NaN,FRANCE,http://senegal-actu.com/2016/03/captured-paris-suspect-was-preparing-terrorist-attacks-in/,201603,12,NaN,85,-4.769915,FR,48.8667,2.33333,senegal-actu.com
271626,2016-03-20,NaN,FRANCE,http://www.theguardian.com/world/2016/mar/20/salah-abdeslam-lawyer-plans-sue-revelations-aborted-suicide-plan,201603,12,NaN,8,-7.415730,FR,48.8667,2.33333,www.theguardian.com
327096,2016-12-10,FRANCE,RUSSIA,http://www.onenewspage.com/n/Front+Page/759ytzu7n/Kazakh-oligarch-Mukhtar-Ablyazov-walks-free-after-France.htm,201612,12,NaN,2,-7.692308,FR,46.0000,2.00000,www.onenewspage.com
335268,2016-03-16,FRANCE,RUSSIA,http://newsok.com/france-kazakh-banker/article/feed/983565,201603,12,NaN,4,-10.259918,FR,48.8667,2.33333,newsok.com


### After reviewing the stories from the URL, they mostly seem to deal with a fairly negative and escalatory action taken by the French government. The Goldstein scale typically gives these stories a `-5` rating based on their WEIS charts, so that's what I will add.

In [20]:
france_events['GoldsteinScale'] = france_events['GoldsteinScale'].fillna(value= -5)

#### Example row where the `GoldsteinScale` was previously empty.

In [25]:
france_events.iloc[[1749977]]

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,DOMAINNAME
1749977,2016-03-16,FRENCH,RUSSIA,http://www.ajc.com/ap/ap/top-news/kazakh-bankers-wife-urges-france-not-to-send-him-t/nqmXb/,201603,12,-5.0,48,-11.01576,FR,48.8667,2.33333,www.ajc.com


In [26]:
missing_zero_values_table(france_events)

Your selected dataframe has 13 columns and 3103885 Rows.
There are 2 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
Actor2Name,0,645666,20.8,645666,20.8,object
Actor1Name,0,212579,6.8,212579,6.8,object


### Part one summary:
 - Formatted the `SQLDATE` to a Pandas datetime object.
 - Extracted the URL from the `SOURCEURL` and gave placed it into the `DOMAINNAME` column
 - Filled in null data for empty `GoldsteinScale` values.

In [27]:
del france_subset

In [28]:
france_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103885 entries, 0 to 3103884
Data columns (total 13 columns):
SQLDATE                  datetime64[ns]
Actor1Name               object
Actor2Name               object
SOURCEURL                object
MonthYear                int64
EventRootCode            int64
GoldsteinScale           float64
NumMentions              int64
AvgTone                  float64
ActionGeo_CountryCode    object
ActionGeo_Lat            float64
ActionGeo_Long           float64
DOMAINNAME               object
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 307.8+ MB


# New Part: Data Wrangling for ML/Logistic Regression Models

## TODO:
- Group by truncated lat/long values to the first decimal place (Completed 7/31)
- Add a column if there was an event the next day or not  (Completed 7/31)
- Train/test split by date (Completed 8/1)
- Build a logistic model (Completed 8/1)
- Fit (Completed 8/1)
- Create a classification report (Completed 8/1)
- Filter by SourceURL and test again
- Iterate over each news source to see if different news sources are more predictive than others

In [98]:
france_events.head()

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,DOMAINNAME,LocationRegion,Event
76287,2016-02-01,IRAN,FRANCE,http://www.presstv.com/Detail/2016/02/01/448257/Iran-mining-foreign-investment-Karbasian,201602,6,6.4,2,1.893939,FR,48.9,2.33333,www.presstv.com,"(48.9, 2.33333)",0
1504333,2016-02-01,PALESTINIAN,ISRAELI,http://www.menafn.com/1094568238/Netanyahu-criticises-French-threat-to-recognise-Palestinian-state,201602,1,0.0,2,-5.244755,FR,46.0,2.00000,www.menafn.com,"(46.0, 2.0)",0
1502271,2016-02-01,NaN,FRANCE,http://iusbpreface.com/2016/02/01/france-to-build-six-n-reactors-in-india-105014/,201602,5,8.0,1,-1.401869,FR,48.9,2.33333,iusbpreface.com,"(48.9, 2.33333)",0
2991617,2016-02-01,BRITISH,COLONEL,http://www.couriermail.com.au/news/world/uk-set-to-deploy-troops-to-libya-to-fight-escalating-islamic-state-terrorism/news-story/8367dd46ab9577db5af2207242db2422?nk=179eeb4d40fb08d89f91b0dda6714c63-1454365043,201602,13,-4.4,34,-5.389052,FR,46.0,2.00000,www.couriermail.com.au,"(46.0, 2.0)",0
1500747,2016-02-01,PARIS,EUROPEAN,http://www.winnfm.com/news/regional/15594-on-historic-foray-castro-visits-france-as-cuba-opens-up,201602,4,1.9,8,0.000000,FR,48.9,2.33333,www.winnfm.com,"(48.9, 2.33333)",0


In [ ]:
decimals = 2    
df['column'] = df['column'].apply(lambda x: round(x, decimals))

### Group by truncated lat/long values

In [24]:
decimals = 1
france_events['ActionGeo_Lat'] = france_events['ActionGeo_Lat'].apply(lambda x: round(x, decimals))

In [ ]:
france_events['ActionGeo_Long'] = france_events['ActionGeo_Long'].apply(lambda x: round(x, decimals))

In [26]:
france_events['LocationRegion'] = list(zip(france_events.ActionGeo_Lat, france_events.ActionGeo_Long))

#### Now we will know how many regions the events are clustered in.

In [28]:
france_events.LocationRegion.nunique()

8710

### Find min and max dates

In [30]:
print (france_events.SQLDATE.min())
print (france_events.SQLDATE.max())

2016-02-01 00:00:00
2019-06-21 00:00:00


### Append ones to dates with an event within this date range.

In [32]:
# Source: https://www.wikiwand.com/en/List_of_terrorist_incidents_in_France#/21st_century
event_list = ['2016-06-12', '2016-07-13', '2016-07-25', '2017-02-02', '2017-03-15', '2017-03-17', '2017-04-19',
             '2017-06-05', '2017-06-18', '2017-08-08', '2017-09-14', '2017-09-30', '2018-03-22', '2018-05-11',
             '2018-12-10', '2019-03-04', '2019-05-23']

In [34]:
france_events['Event'] = 0

### Check if date is one of the dates in the event list

In [93]:
france_events['Event'] = france_events['SQLDATE'].isin(event_list)

### If so, label it a "1"

In [95]:
france_events['Event'] = france_events['Event'].apply(lambda x: 1 if x else 0)

### Check if changes took place

In [86]:
france_events.Event.nunique()

2

In [89]:
start_date = '2018-12-10'
end_date = '2018-12-11'
mask = (france_events['SQLDATE'] > start_date) & (france_events['SQLDATE'] <= end_date)

In [90]:
df = france_events.loc[mask]
df

,SQLDATE,Actor1Name,Actor2Name,SOURCEURL,MonthYear,EventRootCode,GoldsteinScale,NumMentions,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,DOMAINNAME,LocationRegion,Event
97835,2018-12-11,TOKYO,JAPAN,https://www.businessinsider.com/nissans-ceo-calls-renault-heed-carlos-ghosn-allegations-2018-12,201812,11,-2.0,1,-3.429603,FR,46.0,2.000000,www.businessinsider.com,"(46.0, 2.0)",0
98391,2018-12-11,PARIS,POLAND,http://humanitariannews.org/20181218/paris-climate-accord-emission-issue-was-left-untouched,201812,3,4.0,10,2.631579,FR,48.9,2.333330,humanitariannews.org,"(48.9, 2.33333)",0
98405,2018-12-11,GOVERNANCE,JAPAN,https://www.livemint.com/Auto/zXuGDitF7N9WJyDxQCPmGK/Nissan-delays-picking-Carlos-Ghosn-successor-as-tension-brew.html,201812,17,-5.0,2,-4.704097,FR,48.9,2.333330,www.livemint.com,"(48.9, 2.33333)",0
98406,2018-12-11,GOVERNANCE,JAPANESE,https://www.livemint.com/Auto/zXuGDitF7N9WJyDxQCPmGK/Nissan-delays-picking-Carlos-Ghosn-successor-as-tension-brew.html,201812,17,-5.0,6,-4.704097,FR,48.9,2.333330,www.livemint.com,"(48.9, 2.33333)",0
223627,2018-12-11,BRUSSELS,PROTESTER,https://www.upr.org/post/brazilian-president-bolsonaro-withdraws-un-compact-migration,201812,14,-6.5,54,-1.846530,FR,46.0,2.000000,www.upr.org,"(46.0, 2.0)",0
261905,2018-12-11,AFGHANISTAN,NaN,https://www.politico.eu/newsletter/london-playbook/politico-london-playbook-mays-diplomatic-dash-vultures-circle-back-home-monday-in-parliament/,201812,4,1.9,1,-1.251476,FR,48.9,2.333330,www.politico.eu,"(48.9, 2.33333)",0
261906,2018-12-11,GHANA,NaN,http://www.ghananewsagency.org/science/ghana-expects-to-access-more-climate-funding-at-cop24-epa-143079,201812,7,7.4,2,2.272727,FR,48.9,2.333330,www.ghananewsagency.org,"(48.9, 2.33333)",0
261907,2018-12-11,ITALIAN,NaN,https://catholicherald.co.uk/news/2018/12/11/did-angels-really-carry-the-holy-house-of-mary-to-loreto-italy/,201812,4,2.8,10,1.945525,FR,49.1,0.226625,catholicherald.co.uk,"(49.1, 0.226625)",0
261908,2018-12-11,FRENCH,NaN,http://english.ahram.org.eg/NewsContent/2/9/319916/World/International/EU-will-closely-monitor-French-deficit-after-Macro.aspx,201812,1,-0.4,1,-3.341289,FR,46.0,2.000000,english.ahram.org.eg,"(46.0, 2.0)",0
261909,2018-12-11,INTERIOR MINIST,NaN,https://www.reuters.com/article/us-france-shots/at-least-two-dead-11-wounded-in-french-christmas-market-shooting-idUSKBN1OA2A8,201812,1,0.0,64,-6.262439,FR,48.9,2.333330,www.reuters.com,"(48.9, 2.33333)",0


In [96]:
france_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103885 entries, 0 to 3103884
Data columns (total 15 columns):
SQLDATE                  datetime64[ns]
Actor1Name               object
Actor2Name               object
SOURCEURL                object
MonthYear                int64
EventRootCode            int64
GoldsteinScale           float64
NumMentions              int64
AvgTone                  float64
ActionGeo_CountryCode    object
ActionGeo_Lat            float64
ActionGeo_Long           float64
DOMAINNAME               object
LocationRegion           object
Event                    int64
dtypes: datetime64[ns](1), float64(4), int64(4), object(6)
memory usage: 355.2+ MB


### Delete non-numeric values

In [136]:
france_numeric = france_events[['SQLDATE', 'MonthYear', 'EventRootCode', 'GoldsteinScale', 'NumMentions', 'AvgTone','ActionGeo_Lat', 'ActionGeo_Long','Event']]

### Sort values before train/test split

In [137]:
france_numeric = france_numeric.sort_values(by='SQLDATE')

In [ ]:
split_date = pd.datetime(2016,12,20)

df_training = df.loc[df['Date'] <= split_date]
df_test = df.loc[df['Date'] > split_date]

### Create a train/test split

In [138]:
split_date = pd.datetime(2018,12,31)
france_train = france_numeric.loc[france_numeric['SQLDATE'] <= split_date]
france_test = france_numeric.loc[france_numeric['SQLDATE'] > split_date]

### Change SQLDATE to numeric

In [ ]:
pd.to_timedelta(df.time).dt.total_seconds()

In [139]:
france_train['SQLDATE'] = france_train['SQLDATE'].astype(np.int64)
france_test['SQLDATE'] = france_test['SQLDATE'].astype(np.int64)

In [134]:
france_train.shape

(2712843, 9)

In [135]:
france_test.shape

(391042, 9)

### Where the magic (or ML) happens: Use a Logistic Regression classifer to fit the data.

In [140]:
from sklearn.linear_model import LogisticRegression

classifer = LogisticRegression()

classifer.fit(france_train, france_train.Event)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Predict the testing data

In [ ]:
y_pred = clf.predict(X_test)

In [141]:
y_predictions = classifer.predict(france_test)

### Assess the results

In [143]:
from sklearn.metrics import classification_report

print(classification_report(france_test.Event, y_predictions))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99    386522
          1       0.00      0.00      0.00      4520

avg / total       0.98      0.99      0.98    391042



### And that's what you get when less than one percent of your events are from one category.